# ⚛️ Φ‐α Binary Explorer — *one‑click, self‑contained*

This Colab notebook reproduces the **binary‑fraction “peak” search** we’ve been running, but wraps everything in a single function so you can simply hit **“Runtime ▸ Run all”** and get the results.

> **Goal**: find binary fractions `i/2ⁿ` whose powered value `(i/2ⁿ)^POWER` sits **closest** to a chosen target constant (Φ, α, or any numeric target you set).

The notebook then:

1. Enumerates all denominators `2ⁿ` for `n = 1…MAX_BITS`  
2. Computes the absolute difference between `(i/2ⁿ)^POWER` and `TARGET` for every `i` in `[1, 2ⁿ – 1]`  
3. Keeps the **closest candidate per denominator** and returns the global Top‑K across all `n`  
4. Produces a concise table and a `seed_list` of the winning numerators (legacy output you asked for).

Everything is parameterised at the top — tweak and rerun as you like. No manual cell‑ordering mistakes possible.


In [4]:
#@title 🔧 Parameters
# --- Constants -------------------------------------------------
PHI        = (1 + 5 ** 0.5) / 2         # ≈ 1.618033988749895
ALPHA_INV  = 137.035999                 # CODATA 2018
ALPHA      = 1 / ALPHA_INV              # ≈ 7.29735257e‑3

# --- Search hyper‑parameters ----------------------------------
TARGET     = ALPHA * 1e2  #@param {type:"number"}  # <‑‑ change me!
POWER      = 2            #@param {type:"integer"} # raises (i/2ⁿ) to this power
MAX_BITS   = 32           #@param {type:"integer"} # scan n = 1 … MAX_BITS
TOP_K      = 10           #@param {type:"integer"} # how many global best to keep

print(f'Target = {TARGET}')


Target = 0.7297352573756914


In [7]:
#@title 🛠️ Helper functions
from typing import List, Tuple

def best_for_n(n: int, target: float, power: int) -> Tuple[int, float, float]:
    """Return (numerator i, value (i/2ⁿ)^power, abs delta) for the *closest* i."""
    denom = 1 << n  # 2ⁿ
    best_i, best_val, best_delta = None, None, float('inf')
    for i in range(1, denom):  # ignore 0 and 1 for meaningful fractions
        val   = (i / denom) ** power
        delta = abs(val - target)
        if delta < best_delta:
            best_i, best_delta, best_val = i, delta, val
    return best_i, best_val, best_delta

def search(target: float, power: int, max_bits: int) -> List[Tuple[int,int,float,float]]:
    """Return list of (n, i, val, delta) for each denominator 2ⁿ."""
    results = []
    for n in range(1, max_bits + 1):
        i, val, d = best_for_n(n, target, power)
        results.append((n, i, val, d))
    return results

def top_k(results, k):
    return sorted(results, key=lambda t: t[3])[:k]


In [8]:
#@title ▶️ Run search
results = search(TARGET, POWER, MAX_BITS)
top = top_k(results, TOP_K)

import pandas as pd, numpy as np
df = pd.DataFrame(top, columns=['n (bits)', 'i (numerator)', f'(i/2ⁿ)^{POWER}', 'Δ'])
display(df)

# legacy seed_list output (numerators of the top 3)
seed_list = [row[1] for row in top[:3]]
print('\nseed_list =', seed_list)


,n (bits),i (numerator),(i/2ⁿ)^2,Δ
0,32,3668956191,0.729735,1.569233e-10
1,27,114654881,0.729735,2.408658e-10
2,28,229309762,0.729735,2.408658e-10
3,29,458619524,0.729735,2.408658e-10
4,30,917239048,0.729735,2.408658e-10
5,31,1834478096,0.729735,2.408658e-10
6,22,3582965,0.729735,1.248838e-08
7,23,7165930,0.729735,1.248838e-08
8,24,14331860,0.729735,1.248838e-08
9,25,28663720,0.729735,1.248838e-08



seed_list = [3668956191, 114654881, 229309762]
